In [1]:
import rospy
import time 
import numpy as np
import gym

from sensor_msgs.msg import Joy, Image
from webots import WebotsWorld
from webots.krock import Krock
from os import path
from gym import spaces

In [52]:
h = np.array([])

In [56]:
h = np.append(h, np.array([[1,2]]))
h = np.append(h, np.array([[1,2]]))

h[-1]

2.0

In [11]:
h = np.array([[2.9,3,3,3.01,3.01,3.01], [2.9,3,3,3.01,3.01,3.01]])

In [61]:
(np.array([1,2]) <= 0).any()

False

In [63]:
h.diff()

AttributeError: 'numpy.ndarray' object has no attribute 'diff'

In [2]:
rospy.init_node("traversability_simulation")

In [3]:
w = WebotsWorld.from_file(path.abspath('/home/francesco/Documents/Master-Thesis/core/webots/krock/krock2_ros/worlds/krock2_camera.wbt'))

w()

/home/francesco/Documents/Master-Thesis/core/webots/krock/krock2_ros/worlds/krock2_camera.wbt


[WARN] [1541926682.046950]: /krock/supervisor/get_self transport error completing service call: unable to receive data from sender, check sender's logs for details


In [4]:
k = Krock(rate=rospy.Rate(hz=10))

In [5]:
k(w)

In [6]:
actions = spaces.Dict({ 
    'frontal_freq': spaces.Box(low=-1.0, high=1.0, shape=(), dtype=np.float),
    'lateral_freq': spaces.Box(low=-1.0, high=1.0, shape=(), dtype=np.float) 
})

actions.sample()

OrderedDict([('frontal_freq', array(0.09762701)),
             ('lateral_freq', array(0.43037873))])

In [7]:
observation_space = spaces.Dict({
     'sensors':  spaces.Dict({
            'position': spaces.Box(low=-5, high=5, shape=(3,)),
            'velocity': spaces.Box(low=-1, high=1, shape=(3,)),
            'front_cam': spaces.Tuple((
                spaces.Box(low=0, high=1, shape=(600, 800, 3)),
                spaces.Box(low=0, high=1, shape=(600, 800, 3))
            )),
    })
})

observation_space

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


Dict(sensors:Dict(front_cam:Tuple(Box(600, 800, 3), Box(600, 800, 3)), position:Box(3,), velocity:Box(3,)))

In [9]:
k.move(gait=1,
                  frontal_freq=-2,
                  lateral_freq=0,
                  manual_mode=True)

In [10]:
k.stop()

In [10]:
for _ in range(100):
    random_actions = actions.sample()
    k.move(gait=1,
           frontal_freq=random_actions['frontal_freq'], 
           lateral_freq=random_actions['lateral_freq'], 
           manual_mode=True)
    k.sleep()

In [12]:
d = np.zeros(3)

p = k.state['pose'].pose.position

d += [p.x, p.y, p.z]

for i in range(100):
    k.move(1,1,0)
    k.sleep()
    k.stop()
    
    p = k.state['pose'].pose.position

d = [p.x, p.y, p.z] - d
    
print(d)

[0.90564576 0.81774042 0.03148299]


In [ ]:
from webots import WebotsWorld

In [11]:
class KrockWebotsEnv(gym.Env):
    def __init__(self):
        self.w = WebotsWorld()
        self.krock = Krock()
        self.actions =  = spaces.Dict({ 
            'frontal_freq': spaces.Box(low=-1.0, high=1.0, shape=(), dtype=np.float),
            'lateral_freq': spaces.Box(low=-1.0, high=1.0, shape=(), dtype=np.float) 
            })
        self.observation_space = spaces.Dict({
                 'sensors':  spaces.Dict({
                        'position': spaces.Box(low=-5, high=5, shape=(3,)),
                        'velocity': spaces.Box(low=-1, high=1, shape=(3,)),
                        'front_cam': spaces.Tuple((
                            spaces.Box(low=0, high=1, shape=(600, 800, 3)),
                            spaces.Box(low=0, high=1, shape=(600, 800, 3))
                        )),
                })
            })

    def reset(self):
        self.w()
        self.krock()
        
    def step(self, action):
        k.move(gait=1,
           frontal_freq=action['frontal_freq'], 
           lateral_freq=action['lateral_freq'], 
           manual_mode=True)
        k.sleep()
        k.stop()
        
        pose = k.state['pose'].pose.position
        front_cam = k.state['frontal_camera']

SyntaxError: unexpected EOF while parsing (<ipython-input-11-277914b62380>, line 11)

shutdown request: new node registered with same name
